In [1]:
from gamestate import GameState, start_ai_only_game
from models.RandomModel import RandomModel
from models.HeuristicModel import HeuristicModel
from models.DeepRLModel import DeepRLModel
from trainutil import train_models, test_models
import fsutils as fs

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
env = GameState()
deep_rl_model = DeepRLModel()
# rand_model_1 = RandomModel(min_steps=5, max_steps=10)
# rand_model_2 = RandomModel(min_steps=5, max_steps=10)
models = [deep_rl_model]

print(deep_rl_model.model)

DQN(
  (fc1): Linear(in_features=46, out_features=2048, bias=True)
  (fc2): Linear(in_features=2048, out_features=2048, bias=True)
  (fc3): Linear(in_features=2048, out_features=8, bias=True)
  (relu): ReLU()
)


In [4]:
train_models(env, models, episodes=10, steps=2000, print_every=500)
# test_models(env, models, steps=2000)


TRAIN MODE
----STEP 0 rewards----
Model 0: 2.9799999999999898
----STEP 500 rewards----
Model 0: -6.919934147794493
----STEP 1000 rewards----
Model 0: -16.337019265483946
----STEP 1500 rewards----
Model 0: -25.294805326471874
------EPISODE 0 rewards------
Model 0: -33.79907360975926
----STEP 0 rewards----
Model 0: 2.9799999999999898
[15] [VIRUS] Agent1 ate virus 13
----STEP 500 rewards----
Model 0: 112.42770921354654
----STEP 1000 rewards----
Model 0: 109.38389093735503
----STEP 1500 rewards----
Model 0: 106.48852866910575
------EPISODE 1 rewards------
Model 0: 103.73975376659584
----STEP 0 rewards----
Model 0: 0.9799999999999898
----LEARNING BEGINS----
----STEP 500 rewards----
Model 0: -0.1944619011401585
----STEP 1000 rewards----
Model 0: -8.944356902794055
----STEP 1500 rewards----
Model 0: -17.293521685975577
------EPISODE 2 rewards------
Model 0: -22.299298443293083
----STEP 0 rewards----
Model 0: 0.9799999999999898
----STEP 500 rewards----
Model 0: -4.993815392428161
----STEP 100

In [5]:
fs.save_net_to_disk(deep_rl_model.model, "test_drl_10_a")